# Streamlit (HPC)
https://github.com/c00cjz00/RAG_LangChain_streamlit

## 初始環境設定, 切換到 kernel pytorch_2.1.0-cuda11.8-cudnn8-devel.sif

In [ ]:
# 初始環境設定
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin:/usr/ubuntu_bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path

## 確認CUDA版本, 以及否能使用GPU

In [ ]:
!nvidia-smi
import torch
torch.cuda.is_available()

## 可能需要切換到原生 python3 kernel

In [ ]:
%%bash
rm -rf RAG_LangChain_streamlit
git clone https://github.com/c00cjz00/RAG_LangChain_streamlit.git

## 安裝套件, 切換到 kernel pytorch_2.1.0-cuda11.8-cudnn8-devel.sif

In [ ]:
%%bash
cd ./RAG_LangChain_streamlit
pip install -r requirements.txt -q
pip install bitsandbytes==0.42.0 -q

## 強制刪除運行中服務

In [ ]:
# 強制刪除運行中服務
!ps -ef |grep rag_engine.py | awk '{print $2}' | xargs kill -9

## 開啟WebUI
- change rag_engine.py
```
llm = ChatOpenAI(openai_api_key=openai_api_key,model="gpt-3.5-turbo-16k-0613", temperature=0, streaming=True)
to
 llm = ChatOpenAI(openai_api_key=openai_api_key,model="gpt-3.5-turbo-16k-0613", temperature=0, streaming=True)
```
- change rag_engine.py
```
embeddings = HuggingFaceEmbeddings(model_name=model_name,model_kwargs=model_kwargs)
to
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key,model="text-embedding-ada-002")  
```
- change rag_engine.py
```
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
to
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
```

In [ ]:
%%bash
# PATH
cd ./RAG_LangChain_streamlit

# delete older data
rm -rf data/tmp data/vector_store data/vector_store_openai
mkdir data/tmp

# IP
node_ip=$(cat /etc/hosts |grep "$(hostname -a)" | awk '{print $1}')
# PORT
noed_port_genai=$(python -c "import socket; s = socket.socket(socket.AF_INET, socket.SOCK_STREAM); s.bind(('', 0)); addr = s.getsockname(); s.close(); print(addr[1])")
# PROXY
proxy_url=/rstudio/${node_ip}/${noed_port_genai}
# URL
https_url=https://node01.biobank.org.tw${proxy_url}/
echo $https_url

# RUN
streamlit run rag_engine.py --server.address=$(hostname -s) --server.port=${noed_port_genai}
#streamlit run rag_engine.py --server.address=$(hostname -s) --server.port=${noed_port_genai} --server.baseUrlPath=${proxy_url}